#### Ford-Fulkerson algorithm and Edmonds-Karp algorithm

Existing result shows that the maxflow from $s$ to $t$ can be found by iteratively identify an augmenting path and augment the flow. The termination of the algorithm can be proved. This result is used to derive Ford-Fulkerson algorithm

Edmonds and Karp indicates that if, at each iteration, the shorest augmenting path is used to augment, then the algorithm can be proved to have $O(nm^2)$ complexity. The shorest path can be found with BFS.

Edmonds-Karp algorithm is shown below.

In [2]:

import queue

class Edge:
    def __init__(self, fr = -1,to = -1,cap = 0,flow = 0):
        self.fr = fr
        self.to = to
        self.cap = cap
        self.flow = flow


class Edmonds_Karp:
    def __init__(self,n = 0):
        self.n = n 
        self.m = 0
        self.edges = []
        self.G = [[] for _ in range(n)]
        self.vis = [False]*n
        self.dis = [0]*n
        self.pa = []
        self.s = -1
        self.t = -1
    
    def addEdge(self,fr,to,cap):
        self.edges.append(Edge(fr,to,cap,0))
        self.edges.append(Edge(to,fr,0,0))
        self.m = len(self.edges)
        self.G[fr].append(self.m-2)
        self.G[to].append(self.m-1)
    
    def BFS(self):
        self.vis = [False]*self.n
        q = queue.Queue()
        q.put(self.s)
        self.dis[0] = 0
        self.vis[self.s] = True
        self.pa = [None]*self.n
        while not q.empty():
            u = q.get()
            for id in self.G[u]:
                e = self.edges[id]
                if not self.vis[e.to] and e.cap>e.flow:
                    self.vis[e.to] = True
                    self.pa[e.to] = id
                    self.dis[e.to] = self.dis[u]+1
                    q.put(e.to)
        return self.vis[self.t]
    
    def Augment(self):
        a = 1e9
        v = self.t
        while v!=self.s:
            e = self.edges[self.pa[v]]
            a = min(a,e.cap-e.flow)
            v = e.fr
        v = self.t
        while v!=self.s:
            e = self.edges[self.pa[v]]
            e.flow+=a
            self.edges[self.pa[v]^1].flow-=a
            v = e.fr
        return a
    
    def maxflow(self,s,t):
        self.s = s 
        self.t = t
        flow = 0
        while self.BFS():
            flow+=self.Augment()
        return flow

if __name__ == '__main__':
    
    n,m,s,t = map(int,input().split())
    
    dinic = Edmonds_Karp(n)
    
    for i in range(m):
        u,v,c = map(int,input().split())
        dinic.addEdge(u,v,c)
    
    ans = dinic.maxflow(s,t)
    l = [e for e in dinic.edges if e.cap>0 and e.flow>0]
    print(n,ans,len(l))
    for e in l:
        print(e.fr,e.to,e.flow)
    
    

#### Dinic Maxflow Algorithm $O(n^2m)$

Dinic shows that we can do more at each iteration. Instead of finding a single augmenting path, find a blocking $s-t$ flow where the edge is restricted to the level graph constructed by running BFS. 

Intuitively, efficiency is improved by finding more flows instead of a single path so that the shorest path is strictly increasing at each iteration due to blocking property. 


In [ ]:

import queue

class Edge:
    def __init__(self, fr = -1,to = -1,cap = 0,flow = 0):
        self.fr = fr
        self.to = to
        self.cap = cap
        self.flow = flow


class Dinic:
    def __init__(self,n = 0):
        self.n = n 
        self.m = 0
        self.edges = []
        self.G = [[] for _ in range(n)]
        self.vis = [False]*n
        self.dis = [0]*n
        self.cur = [0]*n
        self.s = -1
        self.t = -1
    
    def addEdge(self,fr,to,cap):
        self.edges.append(Edge(fr,to,cap,0))
        self.edges.append(Edge(to,fr,0,0))
        self.m = len(self.edges)
        self.G[fr].append(self.m-2)
        self.G[to].append(self.m-1)
    
    def BFS(self):
        self.vis = [False]*self.n
        q = queue.Queue()
        q.put(self.s)
        self.dis[0] = 0
        self.vis[self.s] = True
        while not q.empty():
            u = q.get()
            for id in self.G[u]:
                e = self.edges[id]
                if not self.vis[e.to] and e.cap>e.flow:
                    self.vis[e.to] = True
                    self.dis[e.to] = self.dis[u]+1
                    q.put(e.to)
        return self.vis[self.t]
    
    def DFS(self,x,a):
        if x==self.t or a==0: return a
        flow = 0
        while self.cur[x]<len(self.G[x]):
            e = self.edges[self.G[x][self.cur[x]]]
            if self.dis[x]+1==self.dis[e.to]:
                f = self.DFS(e.to,min(a,e.cap-e.flow))
                if f>0:
                    e.flow+=f
                    self.edges[self.G[x][self.cur[x]]^1].flow-=f
                    flow+=f
                    a-=f
                    if a==0: break
            
            
            self.cur[x]+=1
        
        return flow
    
    def maxflow(self,s,t):
        self.s = s 
        self.t = t
        flow = 0
        while self.BFS():
            self.cur = [0]*self.n
            flow+=self.DFS(self.s, 1000000000)
        return flow

if __name__ == '__main__':
    
    n,m,s,t = map(int,input().split())
    
    dinic = Dinic(n)
    
    for i in range(m):
        u,v,c = map(int,input().split())
        dinic.addEdge(u,v,c)
    
    ans = dinic.maxflow(s,t)
    l = [e for e in dinic.edges if e.cap>0 and e.flow>0]
    print(n,ans,len(l))
    for e in l:
        print(e.fr,e.to,e.flow)
    
    